In [ ]:
import torch
# The train split of client 1
cl_num = 0
train_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/train.pt')
val_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/val.pt')
test_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/test.pt')
print(len(train_data_client) + len(val_data_client) + len(test_data_client))


1: ESOL(12)
2: FreeSolv(11)
3: Lipophilicity (13)
4: BACE (7)
5: BBBP (8)
6: ClinTox (6)
7: MUTAG (1)
8: PTC_MR (3)
9: PTC_MM (2)
10:  PTC_FM (4)
11: PTC_FR (5) Zu viele Datem: 1440
12: NCI109 (10)
13: NCI1 (9)
14: alchemy_full (15)
15: ZINC_full (16)
16: QM9 (14) (zu viele Daten? 130831)

In [ ]:
len(train_data_client)

In [1]:
import os
import sys

# from federatedscope.core.cmd_args import parse_args
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.auxiliaries.worker_builder import get_client_cls, get_server_cls
from federatedscope.core.configs.config import global_cfg
from federatedscope.core.fed_runner import FedRunner
from yacs.config import CfgNode
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import clear_output


if os.environ.get('https_proxy'):
    del os.environ['https_proxy']
if os.environ.get('http_proxy'):
    del os.environ['http_proxy']

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
2022-12-27 19:30:17,684 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [ ]:
def train(client, lr):
    cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DT/isolated.yaml'
    cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_isolated.yaml'
    #'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

    init_cfg = global_cfg.clone()
    init_cfg.merge_from_file(cfg_file)

    # init_cfg.data.subdirectory = 'graph_dt_backup/processed'
    # init_cfg.merge_from_list(args.opts)
    init_cfg.data.client = client
    init_cfg.train.optimizer.lr = lr
    init_cfg.data.save_dir = 'isolated_theirs'
    update_logger(init_cfg)
    setup_seed(init_cfg.seed)

    # federated dataset might change the number of clients
    # thus, we allow the creation procedure of dataset to modify the global cfg object
    data, modified_cfg = get_data(config=init_cfg.clone())
    init_cfg.merge_from_other_cfg(modified_cfg)

    init_cfg.freeze()

    # allow different settings for different clients
    # cfg_client.merge_from_file(args.cfg_client)
    if cfg_client is None:
        cfg_client = None
    else:
        cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()
    runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=get_client_cls(init_cfg),
                   config=init_cfg.clone(),
                   client_config=cfg_client)
    _ = runner.run()

In [ ]:
clients = range(1, 1+1)
lrs = [0.01]
num_trainings = 3
for client in clients:
    clear_output(wait=True)
    for lr in lrs:
        print(f"Client: {client},\tlr: {lr}")
        for i in range(num_trainings):
            print(f"training run: {i+1}")
            train(client, lr)

In [2]:
cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DT/fedavg.yaml'
cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml'
#'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

init_cfg = global_cfg.clone()
init_cfg.merge_from_file(cfg_file)

# init_cfg.data.subdirectory = 'graph_dt_backup/processed'
# init_cfg.merge_from_list(args.opts)
#init_cfg.data.client = 5
#init_cfg.train.optgraph_level_defaultimizer.lr = 0.01
update_logger(init_cfg)
setup_seed(init_cfg.seed)

# federated dataset might change the number of clients
# thus, we allow the creation procedure of dataset to modify the global cfg object
data, modified_cfg = get_data(config=init_cfg.clone())
init_cfg.merge_from_other_cfg(modified_cfg)

init_cfg.freeze()

# allow different settings for different clients
# cfg_client.merge_from_file(args.cfg_client)
if cfg_client is None:
    cfg_client = None
else:
    cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()



2022-12-27 19:30:18,422 (utils:129)INFO: the current machine is at 127.0.1.1
2022-12-27 19:30:18,423 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2022-12-27 19:30:18,423 (utils:132)INFO: the output dir is exp/Graph-DC-FedAvg_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20221227193018
2022-12-27 19:37:14,205 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim

In [3]:
1


1

In [3]:
runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=get_client_cls(init_cfg),
                   config=init_cfg.clone(),
                   client_config=cfg_client)
_ = runner.run()

2022-12-27 19:37:14,222 (worker_builder:73)WARNING: Server for method Graph-DC-FedAvg is not implemented. Will use default one
2022-12-27 19:37:14,223 (worker_builder:22)WARNING: Clients for method Graph-DC-FedAvg is not implemented. Will use default one
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2022-12-27 19:37:14,550 (aggregator_builder:21)WARNING: Aggregator for method Graph-DC-FedAvg is not implemented. Will use default one
2022-12-27 19:37:14,551 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-12-27 19:37:14,562 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 40

In [ ]:
1

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


In [ ]:
cfg_client['client_5']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_6']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_7']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_8']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_9']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_10']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_11']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_12']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_13']['eval']['metrics']=['mse']

In [ ]:
cl_list = ['client_' + str(i) for i in range(1,17)]

In [ ]:
cl_list

In [ ]:
cfg_client['client_13']['train']['local_update_steps']

In [ ]:
for el in cl_list:
    cfg_client[el]['train']['local_update_steps']=1

In [ ]:
cfg_client